<a href="https://colab.research.google.com/github/mel418/customer-segmentation/blob/main/Customer_Segmentation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Customer Segmentation & Lifetime Value Analysis Project
# Let's start by importing our essential libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("Libraries imported successfully!")
print("Pandas version:", pd.__version__)

Libraries imported successfully!
Pandas version: 2.2.2


In [3]:
# Let's try to load the dataset directly from the web
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

try:
    df = pd.read_excel(url)
    print("✅ Successfully loaded data from UCI!")
except:
    print("❌ UCI link didn't work - we'll use an alternative")
    df = None

✅ Successfully loaded data from UCI!


In [4]:
# lets explore our dataset structure
print("Dataset shape:", df.shape) # Shows (number of rows, number of columns)
print("\nColumn names:") # Lists all column names
print(df.columns.tolist())
print("\nFirst 5 rows:")
df.head() #Shows first 5 rows of data

Dataset shape: (541909, 8)

Column names:
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

First 5 rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Quick Data quality check
print("DATA QUALITY OVERVIEW")
print("="*40)

print(f"Missing values per column:")
missing = df.isnull().sum()
for col, count in missing.items():
    if count > 0:
        print(f"  {col}: {count:,} missing ({count/len(df)*100:.1f}%)")
    else:
        print(f"  {col}: ✅ No missing values")
print(f"\nUnique customers: {df['CustomerID'].nunique():,}")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
print(f"Countries: {df['Country'].nunique()} countries")

DATA QUALITY OVERVIEW
Missing values per column:
  InvoiceNo: ✅ No missing values
  StockCode: ✅ No missing values
  Description: 1,454 missing (0.3%)
  Quantity: ✅ No missing values
  InvoiceDate: ✅ No missing values
  UnitPrice: ✅ No missing values
  CustomerID: 135,080 missing (24.9%)
  Country: ✅ No missing values

Unique customers: 4,372
Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Countries: 38 countries


In [6]:
# Data cleaning strategy
print("BEFORE CLEANING:")
print(f"Total transactions: {len(df):,}")
print(f"Transactions with CustomerID: {df['CustomerID'].notna().sum():,}")
print(f"Transactions without CustomerID: {df['CustomerID'].isna().sum():,}")

# Remove transactions without CustomerID (can't analyze customers without IDs)
df_clean = df[df['CustomerID'].notna()].copy()

print(f"\nAFTER CLEANING:")
print(f"Remaining transactions: {len(df_clean):,}")
print(f"Unique customers: {df_clean['CustomerID'].nunique():,}")
print(f"Data reduction: {(1 - len(df_clean)/len(df))*100:.1f}% removed")

# Let's also check for any other issues
print(f"\nAdditional checks:")
print(f"Negative quantities: {(df_clean['Quantity'] < 0).sum():,}")
print(f"Zero/negative prices: {(df_clean['UnitPrice'] <= 0).sum():,}")

BEFORE CLEANING:
Total transactions: 541,909
Transactions with CustomerID: 406,829
Transactions without CustomerID: 135,080

AFTER CLEANING:
Remaining transactions: 406,829
Unique customers: 4,372
Data reduction: 24.9% removed

Additional checks:
Negative quantities: 8,905
Zero/negative prices: 40


In [8]:
# handle negative quantities and zero prices
print("FINAL DATA CLEANING:")
print("="*30)

print("Before final cleaning:")
print(f"Transactions: {len(df_clean):,}")

# option 1: remove returns/refunds and bad prices for cleaner analysis
df_final = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)].copy()

print(f"\nAfter removing returns and bad prices:")
print(f"Transactions: {len(df_final):,}")
print(f"Customer: {df_final['CustomerID'].nunique():,}")

# create total amount column for each transaction
df_final['TotalAmount'] = df_final['Quantity'] * df_final['UnitPrice']

print(f"\nSample of cleaned data:")
print(df_final[['CustomerID', 'Description', 'Quantity', 'UnitPrice', 'TotalAmount']].head())

print(f"\nData summary:")
print(f"Date range: {df_final['InvoiceDate'].min()} to {df_final['InvoiceDate'].max()}")
print(f"Average transactions value: £{df_final['TotalAmount'].mean():.2f}")
print(f"Total revenue in dataset: £{df_final['TotalAmount'].sum():.2f}")

FINAL DATA CLEANING:
Before final cleaning:
Transactions: 406,829

After removing returns and bad prices:
Transactions: 397,884
Customer: 4,338

Sample of cleaned data:
   CustomerID                          Description  Quantity  UnitPrice  \
0     17850.0   WHITE HANGING HEART T-LIGHT HOLDER         6       2.55   
1     17850.0                  WHITE METAL LANTERN         6       3.39   
2     17850.0       CREAM CUPID HEARTS COAT HANGER         8       2.75   
3     17850.0  KNITTED UNION FLAG HOT WATER BOTTLE         6       3.39   
4     17850.0       RED WOOLLY HOTTIE WHITE HEART.         6       3.39   

   TotalAmount  
0        15.30  
1        20.34  
2        22.00  
3        20.34  
4        20.34  

Data summary:
Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Average transactions value: £22.40
Total revenue in dataset: £8911407.90


In [10]:
# RFM Analysis - customer segmentation
from datetime import datetime

print("Customer Segmentation - RFM Analysis")
print("="*50)

# set analysis date (day after last transaction in dataset)
analysis_date = df_final['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Analysis date: {analysis_date}")

# calculate RFM metrics for each customer
rfm = df_final.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days, # Recency
    'InvoiceNo': 'nunique',                                    # Frequency
    'TotalAmount': 'sum'                                     # Monetary
}).round(2)

# rename columns for clarity
rfm.columns = ['Recency', 'Frequency', 'Monetary']

print(f"\nRFM Summary Statistics:")
print(rfm.describe())

print(f"\nSample RFM data:")
print(rfm.head(10))

Customer Segmentation - RFM Analysis
Analysis date: 2011-12-10 12:50:00

RFM Summary Statistics:
           Recency    Frequency       Monetary
count  4338.000000  4338.000000    4338.000000
mean     92.536422     4.272015    2054.266459
std     100.014169     7.697998    8989.230441
min       1.000000     1.000000       3.750000
25%      18.000000     1.000000     307.415000
50%      51.000000     2.000000     674.485000
75%     142.000000     5.000000    1661.740000
max     374.000000   209.000000  280206.020000

Sample RFM data:
            Recency  Frequency  Monetary
CustomerID                              
12346.0         326          1  77183.60
12347.0           2          7   4310.00
12348.0          75          4   1797.24
12349.0          19          1   1757.55
12350.0         310          1    334.40
12352.0          36          8   2506.04
12353.0         204          1     89.00
12354.0         232          1   1079.40
12355.0         214          1    459.40
12356.0    